In [1]:
import torch
import transformers
from transformers import AutoTokenizer
from time import time
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma

In [2]:
llm_model_name = "meta-llama/Llama-2-7b-chat-hf"
cache_dir = "/home/voldemort/data_science/llm_models"
device = f'cuda:{torch.cuda.current_device()}' if torch.cuda.is_available() else 'cpu'
device

'cuda:0'

In [3]:
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [4]:
model_config = transformers.AutoConfig.from_pretrained(llm_model_name, max_new_tokens=1024)

model = transformers.AutoModelForCausalLM.from_pretrained(
    llm_model_name,
    config=model_config,
    quantization_config=bnb_config,
    device_map=device,
    cache_dir=cache_dir
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
tokenizer = AutoTokenizer.from_pretrained(llm_model_name,cache_dir=cache_dir)
tokenizer.pad_token = tokenizer.eos_token

In [6]:
query_pipeline = transformers.pipeline(
    "text-generation",
    model=model.bfloat16(),
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    max_length=1024,
    device_map="auto"
)

In [10]:
llm = HuggingFacePipeline(pipeline=query_pipeline)

In [11]:
question = "What are the key aspects of Indian Economy?"
response = llm(prompt=question)
print(response)

What are key aspects of Indian Economy?
 everyone knows that the Indian economy is one of the fastest growing economies in the world. It is a mixed economy, with a combination of traditional and modern industries, and a complex system of government policies and regulations. Here are some key aspects of the Indian economy:

1. Size: The Indian economy is the third-largest in Asia and the twelfth-largest in the world by nominal GDP. It has a large and growing middle class, and a rapidly expanding consumer market.

2. Diversification: India has a diverse economy, with a range of industries, including textiles, automobiles, electronics, and pharmaceuticals. It is also a major producer of agricultural products, including rice, wheat, and cotton.

3. Government Policies: The Indian government plays a significant role in the economy through its policies and regulations. It has a complex system of taxation, with a mix of direct and indirect taxes, and a range of subsidies and incentives for va